# Imports

In [1]:
import sys
import os
from molgroups import components as cmp
from molgroups import mol
from molgroups import lipids
import numpy

from bumps.cli import load_model, load_best
from bumps.dream.state import load_state
from molgroups.support import molstat

from IPython.core.magic import register_cell_magic
@register_cell_magic
def write_and_run(line, cell):
    argz = line.split()
    file = argz[-1]
    mode = 'w'
    if len(argz) == 2 and argz[0] == '-a':
        mode = 'a'
    with open(file, mode) as f:
        f.write(cell)
    get_ipython().run_cell(cell)

# Variables

Variables to set the storage folder for the fit, the base name of the runfile, and the global optimizer. Further the
number of burn and production stepst for the MCMC are set. The run flag determines whether the refl1d MCMC is run, or
just a data analysis on a previous fit is performed.

Steps (and burn) are intentionally small in this example to reduce file sizes.

In [2]:
setupdir = os.getcwd()
store = "fit"
runfile = "tblm_protein_hermite"
fitsource = "refl1d"

if not os.path.isdir(store):
    os.mkdir(store)

run = True
burn = 2000
steps = 200


# Example Fit

The content of the following cell is written out to the runfile of the fit 'tblm.py'.

In [3]:
%%write_and_run tblm_protein_hermite.py
# === Import section ===
import numpy
from molgroups import mol
from molgroups import components as cmp
from molgroups import lipids
from refl1d.names import load4, Parameter, SLD, Slab, Experiment, FitProblem
from refl1d.flayer import FunctionalProfile

# === Constant definition section ===
# Canvas
DIMENSION = 450
STEPSIZE = 0.5

# Hermite Spline
CONTROLPOINTS = 8
SPACING = 15.0
PENETRATION = 42
dDp = [None] * CONTROLPOINTS
dVf = [None] * CONTROLPOINTS

# SLDS
PROTDEUT = 1.67e-6
PROTNONDEUT = 3.14e-6
NSLDH2O = -0.5666e-6
NSLDD2O = 6.36e-6

# Define bilayer and protein objects
blm = mol.tBLM(tether=lipids.HC18SAc, filler=cmp.bmeSAc, lipids=[lipids.POPC], lipid_nf=[1.0])
protein = mol.Hermite(10)
protein.numberofcontrolpoints = CONTROLPOINTS


# Bilayer profile definition function
def bilayer(z, sigma, bulknsld, global_rough, rho_substrate, nf_tether, mult_tether, l_tether, l_lipid1, l_lipid2,
            vf_bilayer):
    """ Generic tethered bilayer """

    # Scale all SLDs from Refl1D units (1e-6 Ang^-2) to molgroups units (Ang^-2)
    bulknsld = bulknsld * 1e-6
    rho_substrate = rho_substrate * 1e-6

    blm.fnSet(sigma=sigma, bulknsld=bulknsld, global_rough=global_rough, rho_substrate=rho_substrate,
              nf_tether=nf_tether, mult_tether=mult_tether, l_tether=l_tether, l_lipid1=l_lipid1, l_lipid2=l_lipid2,
              vf_bilayer=vf_bilayer)

    # Calculate scattering properties of volume occupied by bilayer
    normarea, area, nsl = blm.fnWriteProfile(z)

    # Fill in the remaining volume with buffer of appropriate nSLD
    nsld = nsl / (normarea * numpy.gradient(z)) + (1.0 - area / normarea) * bulknsld

    # export objects for post analysis, needs to be from this function
    problem.bilayers = [blm]
    problem.dimension = DIMENSION
    problem.stepsize = STEPSIZE
    problem.moldat = blm.fnWritePar2Dict({}, 'bilayer', numpy.arange(DIMENSION) * STEPSIZE)

    # Return nSLD profile in Refl1D units
    return nsld * 1e6


def bilayer_prot(z, sigma, bulknsld, global_rough, rho_substrate, nf_tether, mult_tether, l_tether, l_lipid1,
                    l_lipid2, vf_bilayer, nf_protein, protexchratio, dDp, dVf):
    """ Generic tethered bilayer """

    # Scale all SLDs from Refl1D units (1e-6 Ang^-2) to molgroups units (Ang^-2)
    bulknsld = bulknsld * 1e-6
    rho_substrate = rho_substrate * 1e-6

    blm.fnSet(sigma=sigma, bulknsld=bulknsld, global_rough=global_rough, rho_substrate=rho_substrate,
              nf_tether=nf_tether, mult_tether=mult_tether, l_tether=l_tether, l_lipid1=l_lipid1, l_lipid2=l_lipid2,
              vf_bilayer=vf_bilayer)

    # Calculate scattering properties of volume occupied by bilayer
    normarea, area, nsl = blm.fnWriteProfile(z)

    protein.fnSetNormarea(normarea)
    protSLD = PROTNONDEUT + protexchratio * (bulknsld-NSLDH2O) / (NSLDD2O-NSLDH2O) * (PROTDEUT-PROTNONDEUT)
    protein.fnSetRelative(SPACING, blm.headgroups2[0].fnGetZ() + 0.5 * 9.56 - PENETRATION, dDp, dVf, protSLD,
                          nf_protein)

    z1 = blm.methylenes1[0].z - 0.5 * blm.methylenes1[0].l
    z2 = blm.methyls1[0].z + 0.5 * blm.methyls1[0].l
    lipidvol = 0
    for methylene in blm.methylenes1:
        lipidvol += methylene.vol
    for methyl in blm.methyls1:
        lipidvol += methyl.vol
    lipidvol *= vf_bilayer
    v1 = protein.fnGetVolume(z1, z2) / lipidvol

    lipidvol = 0
    for methylene in blm.methylenes2:
        lipidvol += methylene.vol
    for methyl in blm.methyls2:
        lipidvol += methyl.vol
    lipidvol *= vf_bilayer
    z1 = blm.methyls2[0].z - 0.5 * blm.methyls2[0].l
    z2 = blm.methylenes2[0].z + 0.5 * blm.methylenes2[0].l
    v2 = protein.fnGetVolume(z1, z2) / lipidvol

    blm.fnSet(sigma=sigma, bulknsld=bulknsld, global_rough=global_rough, rho_substrate=rho_substrate,
              nf_tether=nf_tether, mult_tether=mult_tether, l_tether=l_tether, l_lipid1=l_lipid1, l_lipid2=l_lipid2,
              vf_bilayer=vf_bilayer, hc_substitution_1=v1, hc_substitution_2=v2)

    normarea, area, nsl = blm.fnWriteProfile(z)
    area, nsl = protein.fnOverlayProfile(z, area, nsl, normarea)
    # Fill in the remaining volume with buffer of appropriate nSLD
    nsld = nsl / (normarea * numpy.gradient(z)) + (1.0 - area / normarea) * bulknsld

    # export objects for post analysis, needs to be in this function

    # for plotting best-fits
    problem.bilayers = [blm]
    problem.dimension = DIMENSION
    problem.stepsize = STEPSIZE
    # for statistical analysis of molgroups
    moldict1 = blm.fnWritePar2Dict({}, 'bilayer', numpy.arange(DIMENSION) * STEPSIZE)
    moldict2 = protein.fnWritePar2Dict({}, 'protein', numpy.arange(DIMENSION) * STEPSIZE)
    problem.moldat = {**moldict1, **moldict2}

    # Return nSLD profile in Refl1D units
    return nsld * 1e6


# bilayer parameters
vf_bilayer = Parameter(name='vf_bilayer', value=0.9).range(0.0, 1.0)                #volume fraction bilayer
l_lipid1 = Parameter(name='l_lipid1', value=10.0).range(8, 30)                      #inner methylenes
l_lipid2 = Parameter(name='l_lipid2', value=10.0).range(8, 16)                      #outer methylenes
sigma = Parameter(name='sigma_blm', value=5).range(2, 8)                            #bilayer roughness
global_rough = Parameter(name='sigma_substrate', value=5).range(2, 9)               #substrate roughness
d_oxide = Parameter(name='d_oxide', value=10).range(5, 40)                          #silicon oxide thickness
d_Cr = Parameter(name='d_cr', value=40).range(10, 60)                              #chromium thickness
d_gold = Parameter(name='d_gold', value=100).range(130, 150)                        #gold thickness
rough_cr_au = Parameter(name='sigma_cr_au', value=10).range(1, 15.0)                #chromium-gold interface roughness
nf_tether = Parameter(name='nf_tether', value=0.7).range(0.2, 1.0)                  #number fraction tether
mult_tether = Parameter(name='mult_tether', value=2).range(0.1, 4)                  #bme-to-tether ratio
l_tether = Parameter(name='l_tether', value=10).range(3, 18)

# protein parameters
dl_lipid = Parameter(name='dl_lipid', value=0.0).range(-3., 3.0)                    #change in methylene thickness
vf_bilayer_prot = Parameter(name='vf_bilayer_prot', value=0.9).range(0.0, 1.0)      #volume fraction bilayer w/ protein
nf_protein = 1
protexchratio = 0.8
for i in range(len(dDp)):
    dDp[i] = Parameter(name='dDp'+str(i), value=0.0).range(-1 * SPACING / 3., SPACING / 3.)
for i in range(len(dVf)):
    dVf[i] = Parameter(name='dVf'+str(i), value=0.001).range(-0.001, 0.4)

# === Stack ===
# First, we create a 'material' for each bulk layer, which has an real and imaginary
# scattering length density, stored in a Refl1d object called 'SLD'
d2o = SLD(name='d2o', rho=6.3000, irho=0.0000)
h2o = SLD(name='h2o', rho=-0.56, irho=0.0000)
d2o_prot = SLD(name='d2o_prot', rho=6.3000, irho=0.0000)
h2o_prot = SLD(name='h2o_prot', rho=-0.56, irho=0.0000)
siox = SLD(name='siox', rho=4.1000, irho=0.0000)
silicon = SLD(name='silicon', rho=2.0690, irho=0.0000)
cr = SLD(name='chromium', rho=2.7, irho=0.0)
gold = SLD(name='gold', rho=4.4, irho=0.0)

# Then bulk layers are created, each with its own 'material'.  If you want to force
# two layers to always match SLD you can use the same material in multiple layers.
# The roughnesses of each layer are set to zero to begin with:

layer_d2o = Slab(material=d2o, thickness=0.0000, interface=5.0000)
layer_h2o = Slab(material=h2o, thickness=0.0000, interface=5.0000)
layer_d2o_prot = Slab(material=d2o_prot, thickness=0.0000, interface=5.0000)
layer_h2o_prot = Slab(material=h2o_prot, thickness=0.0000, interface=5.0000)
layer_siox = Slab(material=siox, thickness=d_oxide, interface=global_rough)
layer_silicon = Slab(material=silicon, thickness=0.0000, interface=global_rough)
layer_cr = Slab(material=cr, thickness=d_Cr, interface=rough_cr_au)
layer_gold = Slab(material=gold, thickness=d_gold - (blm.substrate.z + 0.5 * blm.substrate.l), interface=0.0000)

# Use the bilayer definition function to generate the bilayer SLD profile, passing in the relevant parameters.
# Note that substrate and bulk SLDs are linked to their respective materials.
mollayer = FunctionalProfile(DIMENSION * STEPSIZE, 0, profile=bilayer, sigma=sigma, bulknsld=d2o.rho,
                             global_rough=global_rough, rho_substrate=gold.rho, nf_tether=nf_tether,
                             mult_tether=mult_tether, l_tether=l_tether, l_lipid1=l_lipid1, l_lipid2=l_lipid2,
                             vf_bilayer=vf_bilayer)

mollayerh = FunctionalProfile(DIMENSION * STEPSIZE, 0, profile=bilayer, sigma=sigma, bulknsld=h2o.rho,
                              global_rough=global_rough, rho_substrate=gold.rho, nf_tether=nf_tether,
                              mult_tether=mult_tether, l_tether=l_tether, l_lipid1=l_lipid1, l_lipid2=l_lipid2,
                              vf_bilayer=vf_bilayer)
mollayer_prot = FunctionalProfile(DIMENSION * STEPSIZE, 0, profile=bilayer_prot, sigma=sigma, bulknsld=d2o_prot.rho,
                                  global_rough=global_rough, rho_substrate=gold.rho, nf_tether=nf_tether,
                                  mult_tether=mult_tether, l_tether=l_tether, l_lipid1=l_lipid1+dl_lipid,
                                  l_lipid2=l_lipid2+dl_lipid, vf_bilayer=vf_bilayer_prot, nf_protein=nf_protein,
                                  protexchratio=protexchratio, dDp=dDp, dVf=dVf)

mollayerh_prot = FunctionalProfile(DIMENSION * STEPSIZE, 0, profile=bilayer_prot, sigma=sigma, bulknsld=h2o_prot.rho,
                                   global_rough=global_rough, rho_substrate=gold.rho, nf_tether=nf_tether,
                                   mult_tether=mult_tether, l_tether=l_tether, l_lipid1=l_lipid1+dl_lipid,
                                   l_lipid2=l_lipid2+dl_lipid, vf_bilayer=vf_bilayer_prot, nf_protein=nf_protein,
                                   protexchratio=protexchratio, dDp=dDp, dVf=dVf)

# Stack the layers into individual samples, using common layer objects for layers that are unchanged between samples
# Always build the sample from the substrate up. If the neutron beam is incident from the substrate side,
# set back_reflectivity = True in the probe definition later.

sample = layer_silicon | layer_siox | layer_cr | layer_gold | mollayer | layer_d2o
sampleh = layer_silicon | layer_siox | layer_cr | layer_gold | mollayerh | layer_h2o
sample_prot = layer_silicon | layer_siox | layer_cr | layer_gold | mollayer_prot | layer_d2o_prot
sampleh_prot = layer_silicon | layer_siox | layer_cr | layer_gold | mollayerh_prot | layer_h2o_prot

# Set sample parameter ranges and constraints between layer properties, if these are not set using parameters previously

# nSLD parameters
d2o.rho.range(5.8000, 6.4000)
h2o.rho.range(-0.6, 0.6)
d2o_prot.rho.range(5.8000, 6.4000)
h2o_prot.rho.range(-0.6, 0.6)
siox.rho.range(2.7000, 3.80)
cr.rho.range(2.7000, 4.15)
gold.rho.range(4.2000, 4.60)

# === Data files ===
probe = load4('kr095_4column.refl', back_reflectivity=True)
probeh = load4('kr096_4column.refl', back_reflectivity=True)
probe_prot = load4('kr097_4column.refl', back_reflectivity=True)
probeh_prot = load4('kr098_4column.refl', back_reflectivity=True)

# Background parameter
# probe.background.value = 0.0000
probe.background.range(-1e-9, 1e-5)
probeh.background.range(-1e-9, 1e-5)
probe_prot.background.range(-1e-9, 1e-5)
probeh_prot.background.range(-1e-9, 1e-5)
probe.intensity.range(0.95, 1.05)
probeh.intensity = probe.intensity
probe_prot.intensity = probe.intensity
probeh_prot.intensity = probe.intensity
probe.theta_offset.range(-0.001, 0.001)
probeh.theta_offset = probe.theta_offset
probe_prot.theta_offset = probe.theta_offset
probeh_prot.theta_offset = probe.theta_offset
probe.sample_broadening.range(-0.005, 0.02)
probeh.sample_broadening = probe.sample_broadening
probe_prot.sample_broadening = probe.sample_broadening
probeh_prot.sample_broadening = probe.sample_broadening

# Define critical edge oversampling for samples that require it (typically D2O only)
# probe.critical_edge(substrate=silicon, surface=d2o)

# === Problem definition ===
# a model object consists of a sample and a probe,

# step = True corresponds to a calculation of the reflectivity from an actual profile
# with microslabbed interfaces.  When step = False, the Nevot-Croce
# approximation is used to account for roughness.  This approximation speeds up
# the calculation tremendously, and is reasonably accuarate as long as the
# roughness is much less than the layer thickness
step = False

model = Experiment(sample=sample, probe=probe, dz=STEPSIZE, step_interfaces=step)
modelh = Experiment(sample=sampleh, probe=probeh, dz=STEPSIZE, step_interfaces=step)
model_prot = Experiment(sample=sample_prot, probe=probe_prot, dz=STEPSIZE, step_interfaces=step)
modelh_prot = Experiment(sample=sampleh_prot, probe=probeh_prot, dz=STEPSIZE, step_interfaces=step)
problem = FitProblem([model, modelh, model_prot, modelh_prot])

# Running the fit

The following runs the fit using rs.py. It can also be run manually from the command line using, for example:
> refl1d_cli.py tblm.py --fit=dream --init=lhs --parallel --store=fit --burn=16000 --steps=500

In [ ]:
if run:
    if os.path.isfile(os.path.join(store, "sErr.dat")):
        os.remove(os.path.join(store, "sErr.dat"))
    if os.path.isfile(os.path.join(store, "isErr.dat")):
        os.remove(os.path.join(store, "isErr.dat"))
    if os.path.isfile(os.path.join(store, "StatDataPython.dat")):
        os.remove(os.path.join(store, "StatDataPython.dat"))

    fit_object = molstat.CMolStat(fitsource=fitsource, spath=setupdir, mcmcpath=store, runfile=runfile, state=None, problem=None)
    fit_object.Interactor.fnRunMCMC(burn, steps, batch=False)

else:
    print(os.path.join(os.getcwd(), store, runfile))
    res_state = load_state(os.path.join(os.getcwd(), store, runfile))
    res_state.mark_outliers()

    fit_object = molstat.CMolStat(fitsource=fitsource, spath=setupdir, mcmcpath=store, runfile=runfile, state=res_state, problem=problem)


Start value clipped to range for parameter siox rho, d_gold
# -p /Users/frank/Dropbox/My Mac (PN115993.campus.nist.gov)/Documents/programming/molgroups/examples/data_fitting/refl1d/tBLM_protein_hermite/tblm_protein_hermite.py --fit=dream --parallel --init=lhs --store=/Users/frank/Dropbox/My Mac (PN115993.campus.nist.gov)/Documents/programming/molgroups/examples/data_fitting/refl1d/tBLM_protein_hermite/fit --burn=2000 --steps=200 --overwrite --seed=163398
-- Model 0 kr095_4column
.probe
  .back_absorption = back_absorption kr095_4column = 1
  .background = background kr095_4column = 0 in [-1e-09,1e-05]
  .intensity = intensity kr095_4column = 1 in [0.95,1.05]
  .sample_broadening = sample_broadening kr095_4column = 0 in [-0.005,0.02]
  .theta_offset = theta_offset kr095_4column = 0 in [-0.001,0.001]
.sample
  .layers
    [0]
      .interface = sigma_substrate = 5 in [2,9]
      .material
        .irho = silicon irho = 0
        .rho = silicon rho = 2.069
      .thickness = silicon thick

|# Analysis

Statistical analysis of fit results.

Median values, 68% confidence limits (-1, 94%: -2, etc.), and fit intervals for all fit parameters.

In [ ]:
sparse = 0.005
fit_object.fnAnalyzeStatFile(fConfidence=-1, sparse=sparse)

Create bilayer statistics from above parameter statistics. Results are saved to bilayerplotdata.dat.

In [ ]:
fit_object.fnPullMolgroup(['protein'])

In [ ]:
fit_object.fnCreateBilayerPlotData()

Plot selected data from bilayerplotdata.dat

> _cvo is component volume occupancy data [0 ,1] opposed to area profiles [0, area per lipid outer leaflet].

> _corr is corrected data taking into account bilayer material replacement by protein

> cg, phos, and chol subgroups of the PC headgroup are currently only extracted for the first lipid component, to help
with certain applications. They are not scaled down according to the scaling factor of the parent headgroup and
represent the area of one subgroup.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import sys

bilayer_plot_data = pd.read_csv(store + "/bilayerplotdata.dat", sep="\s", engine="python")

zax = bilayer_plot_data["zaxis"]

fig, ax = plt.subplots()
ax.plot(zax, bilayer_plot_data["water_corr_cvo"], label="water", color='lightblue', linewidth=3)
ax.plot(zax, bilayer_plot_data["substrate_corr_cvo"], label="substrate", color='orange', linewidth=3)
ax.plot(zax, bilayer_plot_data["tether_corr_cvo"], label="tether", color='green', linewidth=3)
ax.plot(zax, bilayer_plot_data["innerhg_corr_cvo"] + bilayer_plot_data["outerhg_corr_cvo"], label="headgroups", color='teal', linewidth=3)
#ax.plot(zax, bilayer_plot_data["inner_cg_corr_cvo"] + bilayer_plot_data["outer_cg_corr_cvo"],label="cg")
#ax.plot(zax, bilayer_plot_data["inner_phosphate_corr_cvo"] + bilayer_plot_data["outer_phosphate_corr_cvo"], label="phosphate")
#ax.plot(zax, bilayer_plot_data["inner_choline_corr_cvo"] + bilayer_plot_data["outer_choline_corr_cvo"], label="choline")
ax.plot(zax, bilayer_plot_data["innerhc_corr_cvo"]+bilayer_plot_data["outerhc_corr_cvo"], label="hydrocarbons", color='blue', linewidth=3)
ax.plot(zax, bilayer_plot_data["innerch2_corr_cvo"]+bilayer_plot_data["outerch2_corr_cvo"], label="CH2", color='blue', linewidth=3)
ax.plot(zax, bilayer_plot_data["innerch3_corr_cvo"]+bilayer_plot_data["outerch3_corr_cvo"], label="CH3", color='blue', linewidth=3)
ax.plot(zax, bilayer_plot_data["protein_corr_cvo"], label="protein", color='red', linewidth=3)
ax.plot(zax, bilayer_plot_data["protein_corr_cvo_msigma"], label="", linestyle='dotted', color='red', linewidth=3)
ax.plot(zax, bilayer_plot_data["protein_corr_cvo_psigma"], label="", linestyle='dotted', color='red', linewidth=3)
ax.legend(loc="upper right")
plt.xlabel("Distance (Å)")
plt.ylabel("Component volume occupancy")
ax.minorticks_on()
ax.tick_params(which="both", direction="in", labelsize=10)
ax.tick_params(bottom=True, top=True, left=True, right=True, which="both")
#plt.xlim(0, 100)
#plt.xticks(numpy.arange(-35, 36, 5.0))
plt.grid(True, which='Both')
fig.patch.set_facecolor('white')
ax.figure.set_size_inches(16, 10)
plt.savefig(store + "/cvo", facecolor="white")
plt.show()

Compute statistical properties of interest with 68% confidence limits, such as the area per lipid of the outer lipid
leaflet.

In [ ]:
fit_object.fnCalculateMolgroupProperty(fConfidence=-1)

A simple way to show output files from the fit. Publication-ready graphs can be created from the data in the storage
directory or using fit_object:

# Publication-ready graphs

...

# Useful Routines


In [ ]:
model = load_model(runfile+'.py')
load_best(model, os.path.join(store, runfile)+'.par')
model.chisq_str()

In [ ]:
def plot_components(z, blm):
    dd, aArea, anSL = blm.fnWriteProfile(z)
    aArea = numpy.array(aArea)
    anSL = numpy.array(anSL)
    sel = aArea>0
    anSLD = anSL[sel] / (aArea[sel] * stepsize)

    fig, ax = plt.subplots(3, 1, figsize=(20,24))
    fig.patch.set_facecolor('white')
    ax[0].plot(z[sel], aArea[sel])
    ax[1].plot(z[sel], anSLD)
    ax[2].plot(z[sel], anSL[sel])
    ax[0].set_ylabel('Area')
    ax[1].set_ylabel('nSLD')
    ax[2].set_ylabel('nSL')
    ax[0].set_xlabel('z')
    ax[1].set_xlabel('z')
    ax[2].set_xlabel('z')

    # plot individual components
    for key, val in blm.__dict__.items():       # cycle over all attributes of blm
        if isinstance(val, mol.nSLDObj):        # if instance of nSLDObj, plot its profile
            ddp, aArea, anSL = val.fnWriteProfile(z)
            aArea = numpy.array(aArea)
            anSL = numpy.array(anSL)
            sel = aArea>0
            anSLD = anSL[sel] / (aArea[sel] * stepsize)
            ax[0].plot(z[sel], aArea[sel], '-', label=val.name)
            ax[1].plot(z[sel], anSLD, '-', label=val.name)
            ax[2].plot(z[sel], anSL[sel], '-', label=val.name)

    ax[0].legend(loc=0)
    ax[1].legend(loc=0)
    ax[2].legend(loc=0)

z = numpy.arange(model.dimension) * model.stepsize
stepsize = model.stepsize
blm = model.bilayers[0]

plot_components(z, model.bilayers[0])

In [ ]:
from IPython.display import Image, display
import glob
for file in glob.glob(os.path.join(store, "*.png")):
    display(Image(filename=file))